In [ ]:
from sklearn.metrics import average_precision_score, roc_auc_score
import wandb

from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, IntervalStrategy

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F


Loading the metrics we'll use to evaluate our model's training

In [28]:
auc = evaluate.load("roc_auc")
accuracy = evaluate.load("accuracy")
metric = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precison = evaluate.load("precision")
recall = evaluate.load("recall")

Here we decide with evaluation to use and with dataset to test on

In addition, we load the pre-trained model here

In [74]:
split_type = 'db_no_agree_no_dups'
dataset_name = 'DrugBank'
pretrained_path = "seyonec/PubChem10M_SMILES_BPE_450k"

here we load the dataset, we use train2 since it's the train file that doesn't contain the validation set insode of it.

this ``load_dataset`` method, automatically loads all the files in csv format and creates an HuggingFace's dataset object that is easy to use when fine-tuning models

In [75]:
dataset = load_dataset('csv', data_files={'train': f'split/{split_type}/{dataset_name}/train2.csv',
                                          'validation': f'split/{split_type}/{dataset_name}/val.csv',
                                          'test': f'split/{split_type}/{dataset_name}/test.csv',})

Using custom data configuration default-74e83418e4d2c9a0
Found cached dataset csv (/home/eyal/.cache/huggingface/datasets/csv/default-74e83418e4d2c9a0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

removing uncessencary columns from the dataset

In [77]:
dataset = dataset.rename_column('withdrawn_class', 'labels').\
            remove_columns(['Unnamed: 0', 'index', 'length', 'inchikey', 'groups', 'source']).\
            with_format('torch')

here we load our model and tokenizer, we use the ``AutoModel`` and ``AutoTokenizer`` classes as they provide a generic way to load every model in HuggingFace

In [78]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_path)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_path, num_labels=2,
                                                           id2label={0: 'Not Withdrawn', 1:'Withdrawn'},
                                                           label2id={'Not Withdrawn': 0, 'Withdrawn': 1})

loading configuration file config.json from cache at /home/eyal/.cache/huggingface/hub/models--seyonec--PubChem10M_SMILES_BPE_450k/snapshots/c18fccd09b3326bf2d4633412c256d7db872156d/config.json
Model config RobertaConfig {
  "_name_or_path": "seyonec/PubChem10M_SMILES_BPE_450k",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading file vocab.json from cache at /home/eyal/.cache/huggingface/hub/models

In [79]:
def tokenize_function(examples):
    """this methods tokenize the smiles into ids which are then fed into the transforemr model
    we set the max length of the toknizer to be the longest SMILES in our dataset and pad the rest to this length"""
    return tokenizer(examples["smiles"], padding="max_length", truncation=True, max_length=300)

In [80]:
dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

a method to compute all the metrics we are using to evaluate our models

In [61]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)
    auc_score = auc.compute(prediction_scores=logits[:, 1], references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels)
    aupr = average_precision_score(y_score=logits[:, 1], y_true=labels)
    precision_score = precison.compute(predictions=predictions, references=labels)
    recall_score = recall.compute(predictions=predictions, references=labels)
    return {**f1_score , **{'PR-AUC': aupr}, **accuracy_score, **auc_score, **precision_score, **recall_score}

here we define our entire training arguments
this is a simple HuggingFace object that will contain all the parameters we are using in our training

In [11]:
training_args = TrainingArguments(
    output_dir=f"./results/{split_type}/{dataset_name}/{pretrained_path}",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy=IntervalStrategy.STEPS,
    save_strategy=IntervalStrategy.STEPS,
    report_to='wandb',
    run_name=f'{pretrained_path} {split_type} {dataset_name}',
    logging_steps=50,
    save_steps=50,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


training our model

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset={'Validation': dataset["validation"], 'Test': dataset["test"]},
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()